In [16]:
import h5py
import numpy as np
import os
import pandas as pd
from scipy.interpolate import interp1d

def fill_missing(Y, kind="linear"):
    """Fills missing values independently along each dimension after the first."""

    # Store initial shape.
    initial_shape = Y.shape

    # Flatten after first dim.
    Y = Y.reshape((initial_shape[0], -1))

    # Interpolate along each slice.
    for i in range(Y.shape[-1]):
        y = Y[:, i]

        # Build interpolant.
        x = np.flatnonzero(~np.isnan(y))
        f = interp1d(x, y[x], kind=kind, fill_value=np.nan, bounds_error=False)

        # Fill missing
        xq = np.flatnonzero(np.isnan(y))
        y[xq] = f(xq)
        
        # Fill leading or trailing NaNs with the nearest non-NaN values
        mask = np.isnan(y)
        y[mask] = np.interp(np.flatnonzero(mask), np.flatnonzero(~mask), y[~mask])

        # Save slice
        Y[:, i] = y

    # Restore to initial shape.
    Y = Y.reshape(initial_shape)

    return Y

def process_file(filename):
    with h5py.File(filename, "r") as f:
        dset_names = list(f.keys())
        locations = f["tracks"][:].T
        node_names = [n.decode() for n in f["node_names"][:]]
        
        locations = fill_missing(locations)

tl_index = 0
tr_index = 1
ctr_index = 2
rt_index = 3
rb_index = 4
cbr_index = 5
br_index = 6
bl_index = 7
cbl_index = 8
lb_index = 9
lt_index = 10
ctl_index = 11

tl_loc = locations[:, tl_index, :, :]
tr_loc = locations[:, tr_index, :, :]
ctr_loc = locations[:, ctr_index, :, :]
rt_loc = locations[:, rt_index, :, :]
rb_loc = locations[:, rb_index, :, :]
cbr_loc = locations[:, cbr_index, :, :]
br_loc = locations[:, br_index, :, :]
bl_loc = locations[:, bl_index, :, :]
cbl_loc = locations[:, cbl_index, :, :]
lb_loc = locations[:, lb_index, :, :]
lt_loc = locations[:, lt_index, :, :]
ctl_loc = locations[:, ctl_index, :, :]

geoms = ['tl', 'tr', 'ctr', 'rt', 'rb', 'cbr', 'br', 'bl', 'cbl', 'lb', 'lt', 'ctl']

df = pd.DataFrame()

for geom in geoms:
    x = eval(f"{geom}_loc[:, 0]")
    y = eval(f"{geom}_loc[:, 1]")
    x, y = x.ravel(), y.ravel()
    df[f"{geom}_x"] = x
    df[f"{geom}_y"] = y

# Change the folder to the desired path
folder = r"C:\Users\topohl\Documents\GitHub\sleap\docs\notebooks\testBatch"
file_name = os.path.splitext(os.path.basename(filename))[0]
# Save the csv file in the specified folder with the same name as the h5 file
df.to_csv(os.path.join(folder, f"{file_name}_locs.csv"),




SyntaxError: unexpected EOF while parsing (784767845.py, line 91)

In [13]:
import os

folder = r"C:\Users\topohl\Documents\GitHub\sleap\docs\notebooks\testBatch" # path to the folder containing the files

files = [f for f in os.listdir(folder) if f.endswith(".h5")] # list of h5 files in the folder

for file in files:
    file_path = os.path.join(folder, file) # full path of the file
    process_file(file_path) # call the function to process the file